In [35]:
import json
import pennylane as qml
import pennylane.numpy as np

symbols = ["H", "H", "H"]


def h3_ground_energy(bond_length):
    coordinates = np.array([0, 0, 0, bond_length, 0, 0, bond_length/2, np.sqrt(3)*bond_length/2, 0])

    # Building the molecular hamiltonian for the trihydrogen cation
    hamiltonian, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates, charge=1)

    print(f"qubits: {qubits}")
    hf = qml.qchem.hf_state(electrons=2, orbitals=6)

    # Define the device, using lightning.qubit device
    dev = qml.device("lightning.qubit", wires=qubits)

    @qml.qnode(dev, diff_method="parameter-shift")
    def cost_func(params):
        qml.BasisState(hf, wires=range(qubits))
        qml.DoubleExcitation(params[0], wires=[0, 1, 2, 3])
        qml.DoubleExcitation(params[1], wires=[0, 1, 4, 5])
        return qml.expval(hamiltonian)    
    
    opt = qml.GradientDescentOptimizer(stepsize=0.4)
    params = np.array([0.0, 0.0])

    for n in range(10):
        params, prev_energy = opt.step_and_cost(cost_func, params)
        print(f"--- Step: {n}, Energy: {cost_func(params):.8f}")

    return prev_energy


# These functions are responsible for testing the solution.

def run(test_case_input: str) -> str:
    ins = json.loads(test_case_input)
    outs = h3_ground_energy(ins)
    return str(outs)


def check(solution_output: str, expected_output: str) -> None:
    solution_output = json.loads(solution_output)
    expected_output = json.loads(expected_output)
    assert np.allclose(solution_output,expected_output, atol = 1e-4), "Not the correct ground energy"


# These are the public test cases
test_cases = [
    ('1.5', '-1.232574'),
    ('0.8', '-0.3770325')
]

# This will run the public test cases locally
for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")

Running test case 0 with input '1.5'...
qubits: 6
--- Step: 0, Energy: -1.22373792
--- Step: 1, Energy: -1.22910874
--- Step: 2, Energy: -1.23121801
--- Step: 3, Energy: -1.23204381
--- Step: 4, Energy: -1.23236688
--- Step: 5, Energy: -1.23249326
--- Step: 6, Energy: -1.23254272
--- Step: 7, Energy: -1.23256207
--- Step: 8, Energy: -1.23256964
--- Step: 9, Energy: -1.23257261
Correct!
Running test case 1 with input '0.8'...
qubits: 6
--- Step: 0, Energy: -0.37474085
--- Step: 1, Energy: -0.37667849
--- Step: 2, Energy: -0.37697796
--- Step: 3, Energy: -0.37702422
--- Step: 4, Energy: -0.37703137
--- Step: 5, Energy: -0.37703248
--- Step: 6, Energy: -0.37703265
--- Step: 7, Energy: -0.37703268
--- Step: 8, Energy: -0.37703268
--- Step: 9, Energy: -0.37703268
Correct!
